In [3]:
%matplotlib inline

from consts import *
import os,cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Polygon, Rectangle
from PIL import Image
from keras.models import Model
from joblib import dump, load
import keras
import tensorflow as tf
import json
import glob
import shutil
import time

USE_SVM = False

# directory of images to be tested
img_dir = base_path + 'images/negative_raw_test/'
# file containing selective search data for those images
ss_info_filepath = base_path + 'neg_ss_info.csv'

weights_dir = base_path + 'weights/' + 'cnn_tuned_checkpoint.h5'

log_file_path = base_path + 'rcnn_log.csv'

In [ ]:
# check whether we've already done selective search on images in directory
start_time = time.time()
CHECK_PREV = True

if CHECK_PREV and os.path.isfile(ss_info_filepath):
    df_neg_ss = pd.read_csv(ss_info_filepath)
    paths_list = df_neg_ss['path'].tolist()
else:
    CHECK_PREV = False

ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
ss_file = open(ss_info_filepath, "a+")
    
for fpath in glob.glob(img_dir + '*.*'):    
    if CHECK_PREV:
        if fpath in paths_list:
            continue
    print(fpath)
    # run selective search
    image = cv2.imread(fpath)[...,::-1] # switch to rgb

    # initialise selective search
    ss.setBaseImage(image)
    ss.switchToSelectiveSearchFast()
    ssresults = ss.process()
    print(len(ssresults))
    ss_dict = {'results': np.ndarray.tolist(ssresults)[:2000]}
    
    ss_file.write(fpath + ',' + '\"' + json.dumps(ss_dict) + '\"' + '\n')

ss_file.close()
print(time.time() - start_time)

In [4]:
# load up the CNN weights

from CNN_model import VGG_model, custom_model
if USE_SVM:
    # model.load_weights(weights_dir + 'cnn_' + str(CLASS_ID) + '.h5')
    custom_model.load_weights(weights_dir + 'cnn_0_120_good.h5')
    model = Model(inputs=model.input,
                       outputs=model.layers[-2].output)
else:
    VGG_model.load_weights(weights_dir)
    model = VGG_model


In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 120, 120, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 120, 120, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 120, 120, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 60, 60, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 60, 60, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 60, 60, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 30, 30, 128)       0     

In [ ]:
def resize_norm(crop):
    resized = cv2.resize(crop, (IMG_SIZE,IMG_SIZE), interpolation = cv2.INTER_AREA)
    if not USE_GREY:
        resized_norm = (resized/255) - 0.5
    else:
        resized_norm = resized - 0.5
    resized_norm = np.expand_dims(resized_norm, axis=0)
    return resized_norm

def limit_size(x, y, w, h):
    if x < 0:
        x = 0
    if y < 0:
        y = 0
    if x + w >= im_w:
        w = im_w - x - 1
    if y + h >= im_h:
        h = im_h - y - 1
    return (x, y, w, h)

def extra_test(crop_dims, imout, model):
    x, y, w, h = crop_dims

    probs = []
#     # test an image slightly smaller
#     new_w = int(w*0.95)
#     new_h = int(h*0.95)
#     new_x = x - int((new_w - w)/2)
#     new_y = y - int((new_h - h)/2)
#     new_x, new_y, new_w, new_h = limit_size(new_x, new_y, new_w, new_h)
#     new_crop = imout[new_y:new_y+new_h, new_x:new_x+new_w]
#     probs.append(model.predict(resize_norm(new_crop))[0][1])    

    # test an image even larger
    new_w = int(w*1.05)
    new_h = int(h*1.05)
    new_x = x - int((new_w - w)/2)
    new_y = y - int((new_h - h)/2)
    new_x, new_y, new_w, new_h = limit_size(new_x, new_y, new_w, new_h)
    new_crop = imout[new_y:new_y+new_h, new_x:new_x+new_w]
    probs.append(model.predict(resize_norm(new_crop))[0][1])

    # test an image even larger
    new_w = int(w*1.1)
    new_h = int(h*1.1)
    new_x = x - int((new_w - w)/2)
    new_y = y - int((new_h - h)/2)
    new_x, new_y, new_w, new_h = limit_size(new_x, new_y, new_w, new_h)
    new_crop = imout[new_y:new_y+new_h, new_x:new_x+new_w]
    probs.append(model.predict(resize_norm(new_crop))[0][1])

    # test an image even larger
    new_w = int(w*1.15)
    new_h = int(h*1.15)
    new_x = x - int((new_w - w)/2)
    new_y = y - int((new_h - h)/2)
    new_x, new_y, new_w, new_h = limit_size(new_x, new_y, new_w, new_h)
    new_crop = imout[new_y:new_y+new_h, new_x:new_x+new_w]
    probs.append(model.predict(resize_norm(new_crop))[0][1])  

    # test an image even larger
    new_w = int(w*1.2)
    new_h = int(h*1.2)
    new_x = x - int((new_w - w)/2)
    new_y = y - int((new_h - h)/2)
    new_x, new_y, new_w, new_h = limit_size(new_x, new_y, new_w, new_h)
    new_crop = imout[new_y:new_y+new_h, new_x:new_x+new_w]
    probs.append(model.predict(resize_norm(new_crop))[0][1])

    return max(probs), min(probs)

true_neg_path = img_dir + 'negative/'
false_neg_path = img_dir + 'positive/'

if not os.path.exists(true_neg_path):
    os.mkdir(true_neg_path)
if not os.path.exists(false_neg_path):
    os.mkdir(false_neg_path)

In [ ]:
CHECK_PREV = True
MAKE_PLOT = True

if CHECK_PREV and os.path.isfile(log_file_path):
    df_ss = pd.read_csv(log_file_path)
    names_list = df_ss['name'].tolist()
else:
    CHECK_PREV = False
if CHECK_PREV:
    log_file = open(log_file_path, "a+")
else:
    log_file = open(log_file_path, "w+")


import time
start_time = time.time()
df_neg_ss = pd.read_csv(ss_info_filepath)

for index, row in df_neg_ss.iterrows():
    img_path = row['path'].strip()
    print(img_path)
    if CHECK_PREV:
        fname = img_path.split('/')[-1]
        if fname in names_list:
            continue

    if not os.path.isfile(img_path):
        print("Cant find " + img_path)
        continue
        
    ss_results = json.loads(row['ss_results'])['results']
    image = cv2.imread(img_path)[...,::-1] # switch to rgb
    imout = image

    crop_list = []
    im_h, im_w, _ = image.shape 
    
    for i,result in enumerate(ss_results):
        
        if i >= 2000:
            break
        x,y,w,h = result

        if w < MIN_SIZE or h < MIN_SIZE or h > MAX_SIZE or w > MAX_SIZE:
            continue

        crop = imout[y:y+h,x:x+w]
        resized = cv2.resize(crop, (IMG_SIZE,IMG_SIZE), interpolation = cv2.INTER_AREA)

        if not USE_GREY:
            resized_norm = (resized/255) - 0.5
        else:
            resized_norm = resized - 0.5

        resized_norm = np.expand_dims(resized_norm, axis=0)
        prob = model.predict(resized_norm)
        
        if prob[0][1] > 0.5:
            crop = [x, y, w, h]
            max_prob, min_prob = extra_test(crop, imout, model)
            crop_list.append([crop, prob[0][1], min_prob, max_prob])

    sorted_crop_list = list(sorted(crop_list, key=lambda item:item[2], reverse=True))

    fish_detected = False
    cols = []
    for crop_img in sorted_crop_list:
        crop_dims, prob, min_prob, max_prob = crop_img
        x, y, w, h = crop_dims
        barrier1 = 0.85
        barrier2 = 0.9
        barrier3 = 0.95
        
        max_barrier = 0.98
        if w < 120 and h < 120:
            barrier = barrier3
        elif w < 120 or h < 120:
            barrier = barrier2
        elif w > 500 or h > 500:
            barrier = barrier3
        elif w > 400 or h > 400:
            barrier = barrier2
        else:
            max_barrier = 0.95
            barrier = barrier1
            
        if prob > 0.8 and (min_prob > barrier or max_prob > max_barrier):
            fish_detected = True
            cols.append('red')
        else:
            cols.append('cyan')

    if fish_detected and MAKE_PLOT:
        plt.figure(figsize=(18,18))
        ax = plt.gca()
        ax.imshow(image)
        for col, crop_img in zip(cols, sorted_crop_list):
            crop_dims, prob, min_prob, max_prob = crop_img
            x, y, w, h = crop_dims
            rect = Rectangle([x ,y], w, h, fill=False, color=col, lw=1)
            ax.add_patch(rect)
            prob_str = str(prob)[0:6] 
            prob_str += ', ' + str(min_prob)[0:6]
            prob_str += ', ' + str(max_prob)[0:6]
            if col == 'red':
                z_pos = 10
            else:
                z_pos = 11
            ax.text(x,y + 3, prob_str, c=col, ha='left', va='top', bbox=dict(facecolor='white', alpha=0.25, lw=0, zorder=z_pos))
        save_file = img_dest + fname.split('.')[0] + '(1)' + '.png'
        plt.savefig(save_file, bbox_inches='tight')
        plt.close()

    # log the result and move files to appropriate folders
    fname = img_path.split('/')[-1]
    if fish_detected:
        img_dest = false_neg_path 
    else:
        img_dest = true_neg_path
       
    plt.close()
    # log the results
    dims = [crop_img[0] for crop_img in sorted_crop_list]
    probs = [float(crop_img[1]) for crop_img in sorted_crop_list]
    min_probs = [float(crop_img[2]) for crop_img in sorted_crop_list]
    max_probs = [float(crop_img[3]) for crop_img in sorted_crop_list]
    results_dict = {'dims': dims, 'probs': probs, 'min_probs': min_probs, 'max_probs': max_probs}
    log_line = '{},\"{}\"\n'.format(fname, json.dumps(results_dict))
    log_file.write(log_line)
log_file.close()

print(time.time() - start_time)
